# Triggers

A trigger in SQL is a predefined, automatic action or set of actions that are executed in response to a specific event on a database table. These events can be data modification operations, such as **INSERT, UPDATE, DELETE,** or even certain types of system events. Triggers are a part of DML.

Triggers are typically defined using SQL statements and are attached to specific tables. When the specified triggering event occurs, the trigger's associated actions are executed. These actions can include executing SQL statements, invoking stored procedures, or performing other tasks, depending on the logic defined within the trigger.

## AUTHORS table

### Trigger on `AFTER UPDATE`

This trigger is activated whenever there are done any changes to the data in the Authors table. Previous data is represented and next to the new, updated data is represented.

```sql
CREATE TABLE update_author (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
	old_data VARCHAR(255),
    new_data VARCHAR(255),
    created_at TIMESTAMP);

```SQL
DELIMITER //    
CREATE TRIGGER tr_update_author
AFTER UPDATE ON authors
FOR EACH ROW
BEGIN
	INSERT INTO update_author (user, action, old_data, new_data, created_at)
    VALUES (user(), 'UPDATED', concat_ws(', ', old.author_name, old.birth_date, old.nationality), concat_ws(', ', new.author_name, new.birth_date, new.nationality), now());
END//
DELIMITER ;

Trying out the trigger:
```sql
UPDATE authors
SET author_name = 'Elena Kalashnikov'
WHERE authors_id = 4;

Result:

![update_author_photo](images\update_author.png)

### Trigger on `AFTER DELETE`

Also known as **Soft delete** has a lot of benefits especially in e-commerce world. Main reason for using soft delete trigger in my database is for data recovery. In case a record is deleted accidentally or maliciously, it can be restored because the record is not permanently removed from the database. This can be particularly important for retaining historical data. Soft delete allows for auditing and compliance purposes. By keeping a history of deleted records, organizations can track who deleted records and when they were deleted. Soft delete is useful for archiving and preserving historical data, like users or customers.

```sql
CREATE TABLE delete_author (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
    deleted_data VARCHAR(100),
    deleted_at TIMESTAMP);

```sql
DELIMITER //
CREATE TRIGGER tr_delete_author
AFTER DELETE ON authors
FOR EACH ROW
BEGIN
	INSERT INTO delete_author(user, action, deleted_data, deleted_at)
    VALUES (user(), 'DELETED', concat_ws(', ', old.author_name, old.birth_date, old.nationality), NOW());
END//
DELIMITER ;

```sql
DELETE FROM authors
WHERE authors_id = 219;


Result:

![delete_author_photo](images\delete_author.png)

## BOOKS table

The same logic I applied to Books table, as this is also a strong entity in my database, any change made to it should be tracked, for compliance and also historical purposes.

### Trigger on `AFTER DELETE`

```SQL
CREATE TABLE delete_book (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
    deleted_data VARCHAR(100),
    deleted_at TIMESTAMP);

```SQL
DELIMITER //    
CREATE TRIGGER tr_delete_book
AFTER DELETE ON books
FOR EACH ROW
BEGIN
	INSERT INTO delete_book (user, action, deleted_data, deleted_at)
    VALUES (user(), 'DELETED', concat_ws(', ', old.title, old.publication_date, old.ISBN, old.stock_quantity, old.avarage_rating, old.price), now());
END//
DELIMITER ;


```SQL
DELETE FROM books
WHERE book_id = 160;


Result:

![delete_book_photo](images\delete_book.png)

## Trigger on `AFTER UPDATE`

This trigger is used to send an alert to specified table whenever a book is low in stock. Like this it is easy to control which books needs to be restocked. I didn`t choose to use automatic restocking just in case Bookshop Near Me chooses not to continue a certain book.

## CUSTOMERS table

### Trigger on `AFTER UPDATE`

```SQL
CREATE TABLE update_customer (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
	old_data VARCHAR(255),
    new_data VARCHAR(255),
    created_at TIMESTAMP);


```SQL
 DELIMITER //
CREATE TRIGGER tr_update_customer
AFTER UPDATE ON customers
FOR EACH ROW
BEGIN
	INSERT INTO update_customer (user, action, old_data, new_data, created_at)
    VALUES(user(), 'UPDATED', CONCAT_WS(', ', OLD.first_name, old.last_name, old.phone, old.address, old.membership_status, old.country),
			concat_ws(', ', new.first_name, new.last_name, new.phone, new.address, new.membership_status, new.country ), now());
 END//
 DELIMITER ;


```SQL
 UPDATE customers
SET first_name = 'Ana'
WHERE first_name = 'Ance';


Result:

![update_customers_photo](images\update_customers.png)

### Trigger on `AFTER DELETE`

```SQL
CREATE TABLE delete_customer (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
    deleted_data VARCHAR(100),
    deleted_at TIMESTAMP);


```SQL
DELIMITER //    
CREATE TRIGGER tr_delete_customer
AFTER DELETE ON customers
FOR EACH ROW
BEGIN
	INSERT INTO delete_customer (user, action, deleted_data, deleted_at)
    VALUES (user(), 'DELETED', concat_ws(', ', old.first_name, old.last_name, old.phone, old.address, old.membership_status, old.country), now());
END//
DELIMITER ;


```SQL
DELETE FROM customers
WHERE customer_id = 310;


Result:

![delete_customers_photo](images\delete_customers.png)

## LOYALTY POINTS table

For this table I made a trigger that affects membership status in Customers table. It sums up loyalty points fro specific customer and depending on the amount of loyalty points, membership status is affected or no. 

```sql
DELIMITER //
CREATE TRIGGER update_membership_status 
AFTER UPDATE ON loyaltypoints
FOR EACH ROW
BEGIN
    DECLARE loyalty_points INT;

    SELECT points_earned INTO loyalty_points -- Get the loyalty points earned by the customer 
    FROM loyaltypoints
    WHERE customer_id = NEW.customer_id;

    IF loyalty_points >= 1 AND loyalty_points < 100 THEN -- Update the membership status in the customers table
        UPDATE customers
        SET membership_status = 'Standard'
        WHERE customer_id = NEW.customer_id;
    ELSEIF loyalty_points >= 100 AND loyalty_points < 500 THEN 
        UPDATE customers
        SET membership_status = 'Premium'
        WHERE customer_id = NEW.customer_id;
    ELSEIF loyalty_points >= 500 THEN
        UPDATE customers
        SET membership_status = 'VIP'
        WHERE customer_id = NEW.customer_id;
    END IF;
END//
DELIMITER ;


```sql
UPDATE loyaltypoints
SET points_earned = 101
WHERE loyalty_id = 16;


The changes are reflected in update_customers table

![loyaltypoints_update](images\update_customers.png)

## REVIEWS table

### Trigger on `AFTER INSERT`

Create a trigger whenever a new review is inserted in Reviews table, it affects avarage_rating in Books table. Have to take in account that in Books table, avarage rating is in scalle of 1 - 5, but in Reviews table rating is in the scale of 1 - 10. 

```sql
DELIMITER //
CREATE TRIGGER update_avarage_rating
AFTER INSERT ON reviews
FOR EACH ROW
BEGIN
    DECLARE total_ratings INT;
    DECLARE total_rating_sum DECIMAL(10, 2);
    DECLARE new_average_rating DECIMAL(3, 2);
    
    SELECT COUNT(*) INTO total_ratings -- Calculate the total number of ratings for the book
    FROM reviews
    WHERE book_id = NEW.book_id;
    
    SELECT SUM(rating) INTO total_rating_sum -- Calculate the sum of all ratings for the book
    FROM reviews
    WHERE book_id = NEW.book_id;
    
    SET total_rating_sum = (total_rating_sum / total_ratings) * 0.5; -- Scale the total_rating_sum to a maximum of 5
    
    UPDATE books -- Update the book table with the new average rating
    SET avarage_rating = total_rating_sum
    WHERE book_id = NEW.book_id;
END//
DELIMITER ;


Inserting two new reviews to see if the avarage_rating is really changing:
```sql
INSERT INTO reviews(customer_id, book_id, rating, comments, review_date)
VALUES(5, 2, 10, 'A great trip into unknown' ,now());

INSERT INTO reviews(customer_id, book_id, rating, comments, review_date)
VALUES(6, 2, 1, 'Not worth your time' ,'023-10-24');


## USERS table

### Trigger on `AFTER UPDATE`

```SQL
CREATE TABLE update_user (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
	old_data VARCHAR(255),
    new_data VARCHAR(255),
    updated_at TIMESTAMP
    );


```SQL
DELIMITER //
CREATE TRIGGER tr_update_user
AFTER UPDATE ON users
FOR EACH ROW
BEGIN
	INSERT INTO update_user (user, action, old_data, new_data, updated_at)
    VALUES(user(), 'UPDATED', CONCAT_WS(', ', OLD.email, old.password),
			concat_ws(', ', new.email, new.password), now());
 END//
 DELIMITER ;


```SQL
UPDATE users
SET email = 'ancstridsoko@example.com'
WHERE user_id =309;

Result:

![update_user_photo](images\update_user.png)

### Trigger on `AFTER DELETE`

```SQL
CREATE TABLE delete_user (
	id INT PRIMARY KEY AUTO_INCREMENT,
    user VARCHAR(255),
    action VARCHAR(50),
    deleted_data VARCHAR(100),
    deleted_at TIMESTAMP);


```SQL
DELIMITER //    
CREATE TRIGGER tr_delete_user
AFTER DELETE ON users
FOR EACH ROW
BEGIN
	INSERT INTO delete_user (user, action, deleted_data, deleted_at)
    VALUES (user(), 'DELETED', concat_ws(', ', old.email, old.customer_id), now());
END//
DELIMITER ;


```SQL
DELETE FROM users
WHERE user_id = 309;


Result:

![delete_user_photo](images\delete_user.png)

## ORDER ITEMS table

### Trigger on `AFTER INSERT`

Order Items is a table that is located between entity Books and entity Orders. Order Items in my database is used to list all the items in the Order, their quantity in order and subtotal. It is an optimal way to avoid data redundancy and an easy way to manage the inventory for Books.

This trigger was made to affect Books table stock_quantity column, whenever in Order Items table a specific book appears and it`s quantity of how many copies of that specific book has been bought in that specific order, that affects the total amount of stock_quantity in Books table.

```sql
DELIMITER //
CREATE TRIGGER update_stock_quantity
AFTER INSERT ON orderitems
FOR EACH ROW
BEGIN
    DECLARE book_stock INT;
    
    SELECT stock_quantity INTO book_stock -- Get the current stock_quantity of the book
    FROM books
    WHERE book_id = NEW.book_id;
    
    UPDATE books -- Update the stock_quantity
    SET stock_quantity = book_stock - NEW.quantity
    WHERE book_id = NEW.book_id;
END//
DELIMITER ;


```sql
INSERT INTO orderitems(order_id, book_id, quantity, subtotal)
VALUES(2013, 1, 1, 54.08);
